In [ ]:
#!unzip "/content/drive/MyDrive/Cinsiyet.zip" -d "/content/drive/MyDrive/Cinsiyet"

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# Veri listeleri ve konum
veriler = []
etiket = []
konum = "/content/drive/MyDrive/Cinsiyet/Cinsiyet"

# İşlenecek maksimum resim sayısı
maks_resim_sayisi = 100
sayaç = 0  # İşlenen resim sayacını takip eder

# Verileri yükle ve işleme
for sınıf in os.listdir(konum):
    sınıf_yolu = os.path.join(konum, sınıf)
    if not os.path.isdir(sınıf_yolu):
        continue
    for dizin in os.listdir(sınıf_yolu):
        if sayaç >= maks_resim_sayisi:  # Belirlenen sınırı aşma
            break
        resim_yolu = os.path.join(sınıf_yolu, dizin)
        if os.path.isfile(resim_yolu):
            resim = cv2.imread(resim_yolu)
            if resim is None:
                continue  # Geçersiz resimleri atla
            resim = cv2.resize(resim, (50, 50))
            resim = resim.astype("float32") / 255  # Normalleştir
            veriler.append(resim)
            if sınıf == "Erkek":
                etiket.append(0)
            else:
                etiket.append(1)
            sayaç += 1

# Verileri numpy array'e çevir
veriler = np.array(veriler)
etiket = np.array(etiket)

# Kaydet
np.savez("/content/drive/MyDrive/Classroom/Colab Notebooks/Veriler.npz", veriler)
np.savez("/content/drive/MyDrive/Classroom/Colab Notebooks/Etiketler.npz", etiket)

print(f"İşlenen resim sayısı: {len(veriler)}")





İşlenen resim sayısı: 100


In [9]:
from keras.models import load_model
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Modeli yükleyin
model = load_model("/content/drive/MyDrive/Model/Modelim.h5")

# Görüntüyü okuyun ve ön işleme yapın
resim_yolu = "/content/drive/MyDrive/Cinsiyet/Cinsiyet/Erkek/090544.jpg.jpg"
resim = cv2.imread(resim_yolu)
if resim is None:
    raise FileNotFoundError(f"Resim bulunamadı: {resim_yolu}")

# Görüntüyü yeniden boyutlandır ve normalize et
resim = cv2.resize(resim, (52, 52))
resim = resim.astype("float32") / 255  # Normalize et (0-1 aralığında)
resim = np.expand_dims(resim, axis=0)  # Batch boyutunu ekle

# Modelden tahmin al
tahmin = model.predict(resim)
tahmin_sınıf = np.argmax(tahmin)

# Sonucu yazdır
etiketler = ["Erkek", "Kadın"]  # Modelin sınıflarına uygun etiketler
tahmin_etiketi = etiketler[tahmin_sınıf]

# Görüntüyü göster
plt.imshow(resim[0])
plt.title(f"Tahmin Sınıfı: {tahmin_sınıf} ({tahmin_etiketi})")
plt.axis("off")
plt.show()

print(model.input_shape)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
Tahmin çıktısı: [[3.1486128e-02 2.0445177e-01 2.1151843e-02 5.8038295e-03 2.0714597e-04
  2.5470159e-03 6.1301505e-03 2.8202980e-04 7.2351789e-01 4.4221873e-03]]
Tahmin sınıfı: 8


IndexError: list index out of range